# Team Members

Berker Uğraş and Berkin Öztürk

# Step 1

At first, we've imported torch, matplotlib and pandas libraries. Then we set the device to 'cuda' to use GPU as a hardware accelerator. After that by using ImageFolder interface, we have defined 2 datasets as a train and test data. Also we have created a dummy folder whose name is 'Validation' and put the test data into it since Image Folder Interface needs labels and test data doesn't have. At this step  we have also defined 'TrainDataSplitter' function to split the train data according to their first digits (datasets) to use them in our first Ensemble Model. But, unfortunately, since we have got better loss values and accuracy results with our new Ensemble Model approach (training each model with whole training data-without splitting) we didn't use these split data.

In [1]:
import torch
from torchvision import datasets, transforms
from torch import nn as nn
from torch.nn import functional as F
import math
import matplotlib.pyplot as plt
import pandas as pd
import csv
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using {device} device")

data=pd.read_csv("./CSV_files/sample_1.csv")
print(data)
df=pd.DataFrame(data)


def TrainDataSplitter(data,split):
    indexStorage=[]
    for i in range(0,len(data),1):
        if(data[i][0][18]==split): 
            indexStorage.append(i)
    return indexStorage


train_data=datasets.ImageFolder("./dataset/train", transform= transforms.ToTensor())
test_data=datasets.ImageFolder("./dataset/Validation", transform= transforms.ToTensor())
print(test_data[3])


    
indices0=TrainDataSplitter(train_data.imgs,"0")
indices1=TrainDataSplitter(train_data.imgs,"1")
indices2=TrainDataSplitter(train_data.imgs,"2")
indices3=TrainDataSplitter(train_data.imgs,"3")

Train_mnist=torch.utils.data.Subset(train_data,indices0)
Train_fashmnist=torch.utils.data.Subset(train_data,indices1)
Train_perMnist=torch.utils.data.Subset(train_data,indices2)
Train_perfashMnist=torch.utils.data.Subset(train_data,indices3)


Using cuda device
           id  classification
0      240000               6
1      240001               8
2      240002               9
3      240003               0
4      240004               4
...       ...             ...
39995  279995               2
39996  279996               9
39997  279997               1
39998  279998               4
39999  279999               1

[40000 rows x 2 columns]
(tensor([[[0.8353, 0.8314, 0.7686,  ..., 0.0000, 0.0000, 0.9412],
         [0.0000, 0.7882, 0.8039,  ..., 0.0000, 0.0000, 0.0000],
         [0.6706, 0.0000, 0.7294,  ..., 0.0000, 0.7843, 0.0000],
         ...,
         [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
         [0.4314, 0.8000, 0.0000,  ..., 0.8667, 0.8118, 0.0000],
         [0.6627, 0.8196, 0.0000,  ..., 0.0000, 0.0000, 0.5216]],

        [[0.8353, 0.8314, 0.7686,  ..., 0.0000, 0.0000, 0.9412],
         [0.0000, 0.7882, 0.8039,  ..., 0.0000, 0.0000, 0.0000],
         [0.6706, 0.0000, 0.7294,  ..., 0.0000, 0.7843, 0.0

# Step 2

In this step we've imported DataLoader and TensorDataset libraries from torch.utils.data.

In [18]:
#dataloader
from torch.utils.data import DataLoader

# Submissions

**First Submission**

In the first submission, our approach was creating an ensemble method (each model trained with split train data) that contains to 2 DenseNet Models with different layer sizes (3 and 4) and an MLP with residual connections. After getting pretty bad accuracy results we immediately changed our approach and used only a DenseNet to make predictions.

**Second Submission**

For one of our second submissions, we have used that DenseNet model alone by tuning the layer size, and in another one we tried, we have just used our MLP with Residual Connections model. We didn't use ensemble models for this submission.

**Third Submission**

In our third submission, we have designed a new ensemble model that contains again 2 DenseNets (layer sizes are 4 and 5) and an MLP with residual connections. The difference between the first one is this time we have trained each model by using the whole train data.

# Step 3

In this step, we have defined our DenseNet model. We used this model in our first,second and the third submissions. 


**Transition Layer**

TransitionLayer class contains 3 modules and these are BatchNorm2d (to do Batch Normalization according to the input channels), Conv2d (here, we are doing 1x1 convolution to reduce the channel size to 4) and the final one is AveragePooling2D (we are using this function to do average pooling and accordingly reduce the spatial dimension of the feature map). Finally, I would like to mention about dropRate parameter. Dropout is a regularization method that if dropRate is not equal to zero, each channel will be zeroed out independently (with probability dropRate using samples from a Bernoulli Distribution).


**Dense Layer**

DenseLayer class contains Batch Normalization, Relu as a activation function and finally Conv2d to do convolution. We set the 'padding' of the convolution function to 1 to preserve the spatial dimensions of feature maps. In the forward function part we've applied those modules (BatchnNorm2 ReLU and conv2d) sequentially. Finally we concatenate the output with input (x). We can consider the these operations as the first layer of the dense block. Since there are 3 layers we are going to repeat these operations for 3 times by using the *Dense Block class*.


**Dense Block**

'dense_block' class contains the 'make_layer()' function we use to create a Dense Layers as much as user wants. We do this by creating a 'layers' list and adding the layers defined through the 'Dense Layer' class to the 'layers' list by using a 'for loop'. In the end, we return it as an nn.sequential container by putting the 'layers' list inside of it.


**Dense Net**

'dense_net' class contains dense blocks and transition layers inside of it. In our case, there should have been 2 dense blocks each with 3 layers. So we defined 2 dense blocks and a transition layer using 'dense_block' and 'TransitionLayer' classes, respectively. But before the first dense block, we had to apply 3x3 convolution with 4 output channels and average pooling with stride 2 to reduce the size.  After the first dense block, we should observe an increase in channel sizes since it is dependent to the growth rate and the layer size (4 (channel size after the first convolution) +num_layers*growthRate). So we used a transition layer between dense blocks to reduce the number of channels to 4 (1x1 conv), and 2x2 average pooling with stride 2 to reduce spatial dimensions. In the end, we applied a Fully Connected Layer (Linear) with 10-dimensional output since there are 10 classes. In addition to this, in the last part of the init function we initialized parameters (weight bias) as in the code.

In [19]:

#transitionLayer
class TransitionLayer(nn.Module):
  def __init__(self, in_num, out,dropRate=0.0):
    super(TransitionLayer,self).__init__()
    self.batch=nn.BatchNorm2d(in_num)
    self.conv=nn.Conv2d(in_num,out,kernel_size=1)
    self.avgPool=nn.AvgPool2d(kernel_size=2, stride=2)

    self.dropRate=dropRate
  def forward(self,x):
    x=self.batch(x)
    x=self.conv(x)
    if self.dropRate >0:
      x=F.dropout(x,p=self.dropRate,inplace=False, training=self.training)
    x=self.avgPool(x)

    return x


class DenseLayer(nn.Module):
    def __init__(self, in_planes, out_planes, dropRate=0.0):
        super(DenseLayer, self).__init__()
        self.b1 = nn.BatchNorm2d(in_planes)
        self.r1 = nn.ReLU()
        self.c1 = nn.Conv2d(in_planes, out_planes, kernel_size=3,padding=1) #We are using padding 1 to preserve the spatial dimensions
        self.droprate = dropRate

    def forward(self, x):
        out = self.c1(self.r1(self.b1(x)))
        if self.droprate > 0:
            out = F.dropout(out, p=self.droprate, training=self.training)
        return torch.cat([x, out], 1)

class dense_block(nn.Module):
     def __init__(self, inChannels,num_layers,growthRate,block,dropRate=0.0):
       super(dense_block,self).__init__()
       self.layer = self._make_layer(block, inChannels, growthRate, num_layers, dropRate)
     def _make_layer(self, block, inChannels, growthRate, num_layers, dropRate):
        layers = []
        for i in range(num_layers):
            layers.append(block(inChannels+i*growthRate, growthRate, dropRate))
        return nn.Sequential(*layers)
     def forward(self,x):
        return self.layer(x)




class Dense_net(nn.Module):
    def __init__(self, inChannels,num_layers,growthRate=12,dropRate=0.0):
      super(Dense_net, self).__init__()
      self.c1=nn.Conv2d(inChannels,4,kernel_size=3,padding=1)
      self.avgPool=nn.AvgPool2d(kernel_size=2, stride=2)

      
      self.firstDB=dense_block(4,num_layers,growthRate,DenseLayer,dropRate)
      inChannels=int(4+num_layers*growthRate)
      self.firstTra=TransitionLayer(inChannels,4,dropRate=dropRate) #The 1x1 convolution of the transition layer between the blocks reduces the number of channels back to 4

      #Second Dense Block
      self.secondDB=dense_block(4,num_layers,growthRate,DenseLayer,dropRate)
      inChannels=int(4+num_layers*growthRate)
      self.fc=nn.Linear(inChannels,10)
      self.inChannels=inChannels

      for m in self.modules():
            if isinstance(m, nn.Conv2d):
                n = m.kernel_size[0] * m.kernel_size[1] * m.out_channels
                m.weight.data.normal_(0, math.sqrt(2. / n))
            elif isinstance(m, nn.BatchNorm2d):
                m.weight.data.fill_(1)
                m.bias.data.zero_()
            elif isinstance(m, nn.Linear):
                m.bias.data.zero_()
    
    def forward(self,x):
      x=self.c1(x)
      x=self.avgPool(x)
      # print("after pooling",x.size())
      x=self.firstDB(x)
      # print("after first DB",x.size())
      x=self.firstTra(x)
      # print("after first TRANSITION",x.size())




      x=self.secondDB(x)
      # print("after second DB",x.size())
      x=F.avg_pool2d(x,7)
      # print("after avg",x.size())
      x=x.view(-1,self.inChannels)
      # print("after xview",x.size())
      # print(x)
      x=self.fc(x)
      # print("after fc",x.size())

      return x

# Step 4

In this step, we have prepared our MLP model with residual connections. This model was used in the first, second, and third submissions. In the first submission we used as a member of our first Ensemble Model, in one of our second submissions we used the model alone to make predictions.


**ResLayer**

resLayer class contains Batch Normalization, ReLU as an activation function and dropout with 0.5 probability as a regularization.

**ResBlock**

The 'ResBlock' class contains the 'make_layer()' function that we use to create as many Residual Layers as the user wants.
We do this by creating a 'layers' list and adding the layers defined through the 'resLayer' class to the 'layers' list by  using a 'for loop'. In the end, we return it as an nn.sequential container by putting the 'layers' list inside of it. In the end we have applied batch normalization, ReLU as an activation and a Fully Connected Layer (nn.Linear)

**MLP**

At first step, we have flattened the input data to be able to apply fully connected layer with 3 input features (as the images have got 3 channels) and 256 output features(linear). After the fully connected layer, we stored the output data into the tempX variable. Later, we have applied ResBlock and got the output data. Then we add this output data to the tempX variable and applied batch normalization and ReLU function, respectively. Afterwards, we applied second fully connected layer with 64 output dimensions and then did exactly same operations until the last fully connected layer. Since the data has 10 classes, the last fully connected layer should also have 10 output dimensions. Finally we have applied the last fully connected layer and returned the output data.



In [20]:


class resLayer(nn.Module):
    def __init__(self, inFeatures, dropRate=0.5):
        super(resLayer, self).__init__()
        self.b1=nn.BatchNorm1d(inFeatures)
        self.r1=nn.ReLU()
        self.dp1=nn.Dropout(p=dropRate)
    def forward(self, x):
        x=self.b1(x)
        x=self.r1(x)
        x=self.dp1(x)
        return x

class ResBlock(nn.Module):
  def __init__(self, inFeatures, outFeatures,out2):
    super(ResBlock,self).__init__()
    self.layer=self._make_layer(resLayer,inFeatures,2,0.5)
    self.fc1=nn.Linear(inFeatures,outFeatures)
    self.b2=nn.BatchNorm1d(outFeatures)
    self.r2=nn.ReLU()
    self.fc2=nn.Linear(outFeatures,out2[0])
  def forward(self, x):
    X=self.layer(x)
    x=self.fc1(x)
    x=self.b2(x)
    x=self.r2(x)
    x=self.fc2(x)
    return x

  def _make_layer(self, block, inChannels, num_layers, dropRate):
        layers = []
        for i in range(num_layers):
            layers.append(block(inChannels, dropRate))
        return nn.Sequential(*layers)



class MLP(nn.Module):
    def __init__(self,block):
      super(MLP,self).__init__()
      self.fc1=nn.Linear(28*28*3,256)
      self.resb=block(256,256,[256])
      self.b1=nn.BatchNorm1d(256)
      self.re1=nn.ReLU()
      self.fc2=nn.Linear(256,64)
      self.resb2=block(64,64,[64])
      self.b2=nn.BatchNorm1d(64)
      self.re2=nn.ReLU()
      self.fc3=nn.Linear(64,10)
    def forward(self, x):
      x=torch.flatten(x,1)
      x=self.fc1(x)
      tempX=x
      x=self.resb(x)
      x=x+tempX
      x=self.b1(x)
      x=self.re1(x)

      x=self.fc2(x)
      tempX=x
      x=self.resb2(x)
      x=x+tempX
      x=self.b2(x)
      x=self.re2(x)

      x=self.fc3(x)
      return x

# Step 5

In step 5, we trained and tuned the models we created, and by using these models we made predictions. As it is mentioned before, the ensemble model that we have used in the first and the third submissions are the same, but we used different approaches while training the models of that ensemble model. 

**Ensemble Model**

The function that creates an ensemble model and makes predictions is "EnsembleModelandPred". At the first step, this function adds all the trained models into a list to iterate through each of these. After that, it makes predictions by using our test data for each of these models in the list. Then by using torch.stack we have converted the list into the torch format to be able to sum the prediction values (in axis 0-rows) of each model. In the end, it finds the maximum value on axis 1 (columns) by using the torch.argmax function to be able to select the class which has the highest prediction value.

**First Submission**

In one of our first submissions, we trained model0 (DenseNet 3 layer size), model1(DenseNet 4 layer size), and model2(MLP with residual connections) using the datasets we split earlier. Since the models were trained by using split datasets, the accuracy value wasn't good enough (33%). Then to get a better accuracy value, we applied only our DenseNet model by using the whole train data and got a higher accuracy value (87%). The problem with the first approach was the models of the Ensemble Model were accustomed to their split data (0->MNIST), they hadn't encountered the other types of datasets before (FashionMNIST, permuted MNIST, and permuted FashionMNIST) that's why the prediction values were really unreliable and misleading since we are using a test data which contains all types of datasets. At this point, we decided that we shouldn't split the train data according to their datasets.

**Second Submission**

The first approach that we have used in one of our second submissions is applying DenseNet model (layer size=4) to the whole training data. This approach gave us 88% test accuracy. 

In another second submission, we have applied our MLP with residual connections model to whole training set. And this model gave us 93% test accuracy.

**Third Submission**

For the third submission, we wanted to create a more complex model to increase the test accuracy. Therefore we have decided to use an ensemble model again. But this time we have trained all the models individually by using whole train data. As a result, we have got 94,196% test accuracy, which is a big increase when we look at the accuracy of other models.

**You can find the test and training accuracies of the third submission following:**


**Training accuracy and loss values of the 5-layer DenseNet model:**
![](./NotebookImages/AVGACCURACYLAYERSIZE5.PNG)

![](./NotebookImages/GRAPHLAYER5DENSE.PNG)



**Training accuracy and loss values of the 4-layer DenseNet model:**
![](./NotebookImages/AVGACCURACYLAYERSIZE4DENSE.PNG)

![](./NotebookImages/GRAPHDENSE4.png)



**Training accuracy and loss values of the MLP with Residual Connections:**
![](./NotebookImages/AVGACCURACYMLP.PNG)

![](./NotebookImages/graphMLP.png)




**Validation Accuracy of the Ensemble Model:**
![](./NotebookImages/AccuracyOfEnsemble.PNG)

In [21]:
model0=Dense_net(3,5,12,0).to(device)
model1=Dense_net(3,4,12,0).to(device)
model2=MLP(ResBlock).to(device)

import torch.optim as optim
import torch.nn as nn
import torch

#Training
num_epochs=80
bs=64 #generally powers of 2
lr=0.01

dLoader_train=DataLoader(train_data,bs,shuffle=True)
dLoader_test = DataLoader(test_data)

def EnsembleModelandPred(model0,model1,model2):
    newEnsemble=[model0,model1,model2]
    with torch.no_grad():
        model0.eval()
        model1.eval()
        model2.eval()
        newList=[]
        newEnsemble=[model0,model1,model2]
        for i,(x_batch,y_batch) in enumerate(dLoader_test):
            preds=[model(x_batch.to(device)) for model in newEnsemble]
            preds=torch.stack((preds))
            summed=torch.sum(preds,dim=0)
            ensemble_predictions=torch.argmax(summed,dim=1)
            newList.append(ensemble_predictions.item())
        print(newList)
        df['classification']=newList
        df.to_csv('./CSV_files/testLabelsFinalDeneme.csv',index=False)





#data Loading
# dLoader_train0 = DataLoader(Train_mnist,bs, shuffle=True)
# dLoader_train1 = DataLoader(Train_fashmnist,bs, shuffle=True)
# dLoader_train2 = DataLoader(Train_perMnist,bs, shuffle=True)
# dLoader_train3 = DataLoader(Train_perfashMnist,bs, shuffle=True)


print(len(dLoader_test))



temp_Accuracy=0

#accuracy function
avg_accuracy=0
count=0
def accuracy_func(predictions,target,data_loader,isAvg=0):
    correct=0
    global avg_accuracy
    global count
    global temp_Accuracy
    if(isAvg==1):
        avg_accuracy=avg_accuracy/count
        return avg_accuracy*100
    
    if(count==(len(data_loader)*num_epochs)):   
        temp_Accuracy=0
        avg_accuracy=0
        count=0
    for i in range(0,len(target),1):
        if(target[i]==predictions[i]):
            correct=correct+1
    accuracy=correct/len(target)
    temp_Accuracy=accuracy
    avg_accuracy+=accuracy
    count=count+1
  
def training(model,data_loader):    
    losses=[]
    accuracy_train=[]    
    #Loss
    loss = nn.CrossEntropyLoss()

    #optimizer
    optimizer=optim.SGD(model.parameters(),lr)
    #training data
    model.train()
    for epoch in range(num_epochs):
        ll=0
        #for training
        for i,(x_batch,y_batch) in enumerate(data_loader):
            optimizer.zero_grad()
            y_hat=model(x_batch.to(device))
            _,predicted=torch.max(y_hat.data,1)   
            accuracy_func(predicted,y_batch.to(device),data_loader)
            lossVal=loss(y_hat,y_batch.to(device))  
            lossVal.backward()
            optimizer.step()
            ll +=lossVal

        print(f'loss training {lossVal}')
        losses.append(ll.item()/len(data_loader))
        accuracy_train.append(temp_Accuracy*100)

    print("Average Accuracy of training data",accuracy_func(0,0,0,1),"%")
    plt.figure(figsize=(16,6))
    plt.subplot(1,3,1)
    plt.title("training loss")
    plt.plot(losses[:])
    plt.ylabel('loss')
    plt.xlabel('# of epochs')

    plt.subplot(1,3,2)
    plt.title("training accuracy")
    plt.plot(accuracy_train[:])
    plt.ylabel('accuracy')
    plt.xlabel('# of epochs')

    
#Since we dont have a great hardware system we saved the models after every training procedures

# training(model0,dLoader_train)
# torch.save(model0.state_dict(),"./saved_models/denseNetMDL1_5.pth")
# print("model0 saved")

# training(model1,dLoader_train)
# torch.save(model1.state_dict(),"./saved_models/denseNetMDL2.pth")
# print("model1 saved")

# training(model2,dLoader_train)
# torch.save(model2.state_dict(),"./saved_models/MLPMDL3.pth")
# print("model2 saved")




model0.load_state_dict(torch.load("./saved_models/denseNetMDL1_5.pth"))
model1.load_state_dict(torch.load("./saved_models/denseNetMDL2.pth"))
model2.load_state_dict(torch.load("./saved_models/MLPMDL3.pth"))

EnsembleModelandPred(model0,model1,model2)



40000
[6, 9, 0, 8, 2, 9, 6, 0, 7, 4, 2, 3, 2, 9, 7, 1, 1, 0, 1, 2, 6, 9, 1, 6, 0, 2, 2, 6, 0, 3, 0, 2, 5, 3, 7, 0, 2, 5, 5, 3, 2, 0, 5, 9, 8, 7, 9, 8, 7, 3, 4, 1, 2, 0, 9, 7, 5, 4, 4, 8, 7, 4, 0, 3, 0, 4, 3, 4, 2, 6, 9, 1, 8, 2, 4, 7, 1, 9, 2, 3, 3, 0, 8, 7, 1, 9, 4, 6, 1, 1, 8, 5, 3, 2, 9, 4, 4, 1, 5, 1, 1, 8, 4, 3, 7, 3, 3, 9, 0, 3, 1, 1, 5, 0, 6, 6, 7, 8, 0, 5, 7, 7, 8, 5, 3, 0, 9, 5, 7, 7, 2, 2, 8, 8, 5, 7, 2, 9, 4, 8, 4, 5, 3, 7, 6, 7, 2, 0, 1, 3, 2, 8, 4, 9, 6, 9, 5, 5, 0, 6, 6, 0, 2, 1, 2, 1, 3, 2, 1, 4, 9, 2, 5, 9, 9, 0, 8, 3, 1, 8, 7, 7, 9, 8, 9, 9, 2, 9, 6, 7, 0, 4, 8, 2, 9, 2, 2, 5, 5, 2, 7, 6, 1, 6, 6, 7, 3, 6, 2, 6, 2, 9, 0, 4, 2, 9, 9, 6, 0, 1, 2, 3, 0, 8, 0, 6, 8, 5, 9, 4, 9, 0, 3, 1, 5, 1, 9, 9, 3, 3, 6, 3, 4, 4, 4, 5, 3, 8, 8, 6, 9, 5, 6, 0, 1, 5, 7, 0, 7, 4, 7, 4, 8, 1, 9, 2, 2, 5, 6, 0, 0, 7, 0, 4, 4, 3, 6, 5, 1, 8, 1, 4, 1, 5, 2, 3, 3, 5, 4, 8, 1, 3, 9, 2, 6, 5, 5, 8, 2, 2, 9, 6, 8, 6, 1, 2, 2, 2, 1, 2, 1, 9, 7, 0, 6, 5, 8, 5, 2, 0, 0, 0, 7, 4, 7, 5, 2, 3, 8, 3, 5, 